## Setup

In [1]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd()))

from sqlalchemy import create_engine
from utils import get_session
from data.models.eventhub import Ticket
from agentic.agents.classifier import TicketClassifier
from agentic.agents.resolver import TicketResolver
from agentic.agents.escalation import EscalationAgent, print_escalation_for_human
from agentic.tools.rag_tools import search_knowledge_base
from agentic.tools.db_tools import get_user_info, get_reservation_info

# Create database engine
DB_PATH = "data/db/eventhub.db"
engine = create_engine(f"sqlite:///{DB_PATH}")

## Initialize Agents

In [2]:
# Initialize all agents
classifier = TicketClassifier()
resolver = TicketResolver()
escalation_agent = EscalationAgent()

print("✅ All agents initialized")
print(f"  Classifier: {classifier.model_id}")
print(f"  Resolver: {resolver.model_id}")
print(f"  Escalation: {escalation_agent.model_id}")

✅ All agents initialized
  Classifier: us.meta.llama3-3-70b-instruct-v1:0
  Resolver: us.meta.llama3-3-70b-instruct-v1:0
  Escalation: us.meta.llama3-3-70b-instruct-v1:0


## Helper Function: Fetch Tool Results

In [3]:
def fetch_tool_results(ticket_data, classification):
    """Fetch tool results based on ticket category (CultPass pattern)."""
    tool_results = {}
    
    # Build query
    query = f"{ticket_data.get('subject', '')} {ticket_data.get('description', '')}"
    
    # Always search knowledge base
    kb_results = search_knowledge_base.invoke({"query": query, "top_k": 3})
    tool_results["kb_results"] = kb_results
    
    category = classification.get("category", "general")
    
    # Category-specific tool calling
    if category in ["refund", "cancellation", "complaint"]:
        if ticket_data.get("user_email"):
            try:
                tool_results["user_info"] = get_user_info.invoke({"user_email": ticket_data["user_email"]})
            except Exception as e:
                tool_results["user_info"] = f"Error: {str(e)}"
        
        if ticket_data.get("reservation_id"):
            try:
                tool_results["reservation_info"] = get_reservation_info.invoke({"reservation_id": ticket_data["reservation_id"]})
            except Exception as e:
                tool_results["reservation_info"] = f"Error: {str(e)}"
    
    elif category == "general":
        if ticket_data.get("user_email"):
            try:
                tool_results["user_info"] = get_user_info.invoke({"user_email": ticket_data["user_email"]})
            except:
                pass
    
    return tool_results

## Create Test Complaint Ticket (Always Escalates)

In [4]:
# Create a complaint ticket that will be escalated
complaint_ticket = {
    "ticket_id": "TKT-TEST-001",
    "subject": "Terrible experience - I want my money back!",
    "description": """I am extremely frustrated with your service! I booked tickets for the jazz 
    concert last week and when I arrived, the event was CANCELLED without any notification. 
    I drove 2 hours to get there and wasted my entire evening. This is completely unacceptable!
    I demand a full refund AND compensation for my travel expenses. I've been a loyal customer
    for 3 years and this is how you treat me? I'm considering cancelling my premium subscription!""",
    "category": "complaint",
    "user_email": "john.doe@email.com",
    "reservation_id": "RES001",
    "status": "open"
}

print("Test Ticket Created:")
print(f"Subject: {complaint_ticket['subject']}")
print(f"Category: {complaint_ticket['category']}")

Test Ticket Created:
Subject: Terrible experience - I want my money back!
Category: complaint


## Full Flow: Classify → Fetch Tools → Resolve → Escalate

In [5]:
# Step 1: Classify
print("STEP 1: Classification")
print("="*60)
classification = classifier.classify(
    subject=complaint_ticket['subject'],
    description=complaint_ticket['description']
)
print(f"Category: {classification['category']}")
print(f"Urgency: {classification['urgency']}")
print(f"Sentiment: {classification['sentiment']}")
print(f"Summary: {classification['summary']}")

STEP 1: Classification
Category: refund
Urgency: high
Sentiment: negative
Summary: Customer demands refund and compensation after arriving at a cancelled event without notification
Category: refund
Urgency: high
Sentiment: negative
Summary: Customer demands refund and compensation after arriving at a cancelled event without notification


In [6]:
# Step 2: Fetch tool results
print("\nSTEP 2: Fetch Tool Results")
print("="*60)
tool_results = fetch_tool_results(complaint_ticket, classification)
print(f"Tools called: {list(tool_results.keys())}")


STEP 2: Fetch Tool Results
Tools called: ['kb_results', 'user_info', 'reservation_info']
Tools called: ['kb_results', 'user_info', 'reservation_info']


In [7]:
# Step 3: Attempt resolution
print("\nSTEP 3: Attempt Resolution")
print("="*60)
resolution = resolver.resolve(
    ticket_data=complaint_ticket,
    classification=classification,
    tool_results=tool_results
)
print(f"Status: {resolution['status']}")
print(f"RAG Confidence: {resolution['rag_confidence']:.1%}")

if resolution['status'] == 'escalated':
    print(f"Escalation Reason: {resolution['escalation_reason']}")


STEP 3: Attempt Resolution
Status: escalated
RAG Confidence: 0.0%
Escalation Reason: Low knowledge base confidence (0.0%)
Status: escalated
RAG Confidence: 0.0%
Escalation Reason: Low knowledge base confidence (0.0%)


In [8]:
# Step 4: Handle escalation
if resolution['status'] == 'escalated':
    print("\nSTEP 4: Escalation Agent Processing")
    print("="*60)
    
    escalation_result = escalation_agent.escalate(
        ticket_data=complaint_ticket,
        classification=classification,
        tool_results=tool_results,
        escalation_reason=resolution['escalation_reason'],
        resolver_response=resolution.get('response')
    )
    
    print(f"Escalation processed. Priority: {escalation_result['priority']}")
else:
    print("\n✅ Ticket was resolved, no escalation needed.")


STEP 4: Escalation Agent Processing
Escalation processed. Priority: 2
Escalation processed. Priority: 2


## Display Escalation for Human Agent

In [9]:
# Print formatted escalation details for human agent
if resolution['status'] == 'escalated':
    print_escalation_for_human(escalation_result)
else:
    print("No escalation to display.")

🚨 ESCALATED TICKET - HUMAN AGENT REQUIRED

📊 PRIORITY: 🟠 HIGH
   Reason: Low knowledge base confidence (0.0%)

────────────────────────────────────────
📋 TICKET INFORMATION
────────────────────────────────────────
  Ticket ID: TKT-TEST-001
  Subject: Terrible experience - I want my money back!
  User Email: john.doe@email.com
  Reservation ID: RES001

  Description:
  I am extremely frustrated with your service! I booked tickets for the jazz 
    concert last week and when I arrived, the event was CANCELLED without any notification. 
    I drove 2 hours to get there and wasted my entire evening. This is completely unacceptable!
    I demand a full refund AND comp...

────────────────────────────────────────
🏷️ CLASSIFICATION
────────────────────────────────────────
  Category: REFUND
  Urgency: HIGH
  Sentiment: NEGATIVE
  Summary: Customer demands refund and compensation after arriving at a cancelled event without notification

────────────────────────────────────────
🔧 CONTEXT GATHER

## Test with Real Database Ticket

In [ ]:
# Load a real ticket from database
with get_session(engine) as session:
    # Try to find a complaint or refund ticket
    real_ticket_query = session.query(Ticket).filter(
        Ticket.category.in_(['complaint', 'refund', 'cancellation'])
    ).first()
    
    if real_ticket_query:
        real_ticket = {
            'ticket_id': real_ticket_query.ticket_id,
            'subject': real_ticket_query.subject,
            'description': real_ticket_query.description,
            'category': real_ticket_query.category,
            'user_email': real_ticket_query.user_email,
            'reservation_id': real_ticket_query.reservation_id,
            'status': real_ticket_query.status,
        }
        print(f"Loaded ticket: {real_ticket['ticket_id']}")
        print(f"Category: {real_ticket['category']}")
        print(f"Subject: {real_ticket['subject']}")
    else:
        print("No complaint/refund tickets found in database")
        real_ticket = None

In [ ]:
# Process real ticket through full flow
if real_ticket:
    # Classify
    classification = classifier.classify(
        subject=real_ticket['subject'],
        description=real_ticket['description']
    )
    
    # Fetch tools
    tool_results = fetch_tool_results(real_ticket, classification)
    
    # Resolve
    resolution = resolver.resolve(
        ticket_data=real_ticket,
        classification=classification,
        tool_results=tool_results
    )
    
    print(f"\nResolution Status: {resolution['status']}")
    
    # Escalate if needed
    if resolution['status'] == 'escalated':
        escalation_result = escalation_agent.escalate(
            ticket_data=real_ticket,
            classification=classification,
            tool_results=tool_results,
            escalation_reason=resolution['escalation_reason'],
            resolver_response=resolution.get('response')
        )
        
        # Display for human
        print_escalation_for_human(escalation_result)
    else:
        print(f"\n✅ Resolved: {resolution['response'][:200]}...")

## Summary

The Escalation Agent follows the **CultPass pattern**:

- ✅ Receives all context from Resolver (no additional tool calls)
- ✅ Single LLM call for customer message
- ✅ Formats tool results for human agent display
- ✅ Calculates priority based on classification
- ✅ Provides complete context package for human handoff

**Information Passed to Human Agent:**
- Ticket details (ID, subject, description)
- Classification (category, urgency, sentiment)
- Escalation reason and priority
- KB articles searched
- User information (if fetched)
- Reservation information (if fetched)
- Customer message sent

**Next Steps:**
1. Build LangGraph workflow
2. Connect all agents in proper flow
3. Add state management